# Bio2 Benchmark

We observe trajectories of the Spring pendulum adapted from a biological system and we attempt to learn the system dynamics using the `auto_koopman` function. We generate a training set by sampling $n$ random states uniformly and simulating the system over a time horizon. We learn the system, and test against a new initial value.

In [1]:
# the notebook imports
import matplotlib.pyplot as plt
import numpy as np
# this is the convenience function
from autokoopman import auto_koopman
# for a complete example, let's create an example dataset using an included benchmark system
import autokoopman.benchmark.bio2 as bio2

from sklearn.metrics import mean_squared_error
import statistics
import os
import csv

import time

In [2]:
bio2 = bio2.Bio2()

In [3]:
def get_trajectories(iv):
    # get the model from the experiment results
    model = experiment_results['tuned_model']

    # simulate using the learned model
    trajectory = model.solve_ivp(
        initial_state=iv,
        tspan=(0.0, 10.0),
        sampling_period=0.1
    )
    # simulate the ground truth for comparison
    true_trajectory = bio2.solve_ivp(
        initial_state=iv,
        tspan=(0.0, 10.0),
        sampling_period=0.1
    )
    
    return trajectory, true_trajectory

In [4]:
def test_trajectories(dim,low, high,num_tests):
    mses = []
    mses_dim = [[] for x in range(dim)]
    for j in range(num_tests):
        iv = np.random.uniform(low=low, high=high, size=(dim))
        trajectory, true_trajectory = get_trajectories(iv)
        print(trajectory.states.T)
        mse = mean_squared_error(trajectory.states.T, true_trajectory.states.T)
        mses.append(mse)
        
        for traj_dim, (trajectory, true_trajectory) in enumerate(zip(trajectory.states.T, true_trajectory.states.T)):
            mse = mean_squared_error(trajectory, true_trajectory)
            mses_dim[traj_dim].append(mse)
            
    return mses, mses_dim

In [5]:
def store_data(bench_name, dim, param_name, param_values, train_times, all_mses, all_mses_dim):
    if not os.path.exists('data'):
        os.makedirs('data')
        
    with open(f'data/{bench_name}', 'a') as f:
        writer = csv.writer(f)
        row = [param_name, "train_time", "Avg mse"]
        for i in range(dim):
              row.append(f'Avg mse dim {i+1}')
        writer.writerow(row)
              
        for param_value, train_time, mse, mses_dim in zip(param_values, train_times, all_mses, all_mses_dim):
              row = [param_value, train_time, mse]
              for mses in mses_dim:
                  row.append(mses)
              writer.writerow(row)
        writer.writerow([])

In [6]:
def plot(trajectory, true_trajectory):
    plt.figure(figsize=(10, 6))
    # plot the results
    plt.plot(trajectory.states.T[0], trajectory.states.T[1],label='Trajectory Prediction')
    plt.plot(true_trajectory.states.T[0], true_trajectory.states.T[1],label='Ground Truth')

    plt.xlabel("$x_1$")
    plt.ylabel("$x_2$")
    plt.grid()
    plt.legend()
    plt.title("Bio2 Test Trajectory Plot")
    plt.show()

In [7]:
dim = 9
low = 0.99
high = 1.01
param_values = []
all_mses = []
all_mses_dim = []
param_values = []
times = []
for i in range(1):
    np.random.seed(0)
    start = time.time()
    # generate training data
    training_data = bio2.solve_ivps(initial_states=np.random.uniform(low=low, high=high, size=(10, dim)),
    tspan=[0.0, 10.0], sampling_period=0.1)
    # learn model from data
    experiment_results = auto_koopman(
        training_data,          # list of trajectories
        sampling_period=0.1,    # sampling period of trajectory snapshots
        obs_type="rff",         # use Random Fourier Features Observables
        opt="grid",             # grid search to find best hyperparameters
        n_obs=200,              # maximum number of observables to try
        max_opt_iter=200,       # maximum number of optimization iterations
        grid_param_slices=5,   # for grid search, number of slices for each parameter
        n_splits=5,             # k-folds validation for tuning, helps stabilize the scoring
        rank=(1, 200, 40)       # rank range (start, stop, step) DMD hyperparameter
    )
    end = time.time()
    times.append(round(end - start, 3))
    
    mses, mses_dim = test_trajectories(dim, low, high, 10)
    print("The average mean square error is ", statistics.mean(mses))
    all_mses.append(statistics.mean(mses))
    all_mses_dim.append([])
    for traj_dim, mses in enumerate(mses_dim):
        all_mses_dim[i].append(statistics.mean(mses))
        print(f"The average mean square error for dim {traj_dim+1} is", statistics.mean(mses))
        
#store_data("bio2", dim, "grid_param_slices", param_values, times, all_mses, all_mses_dim)

  0%|                                                    | 0/25 [00:00<?, ?it/s]


NameError: name 'Observable' is not defined